# Battleships

## Planning

- standard version & extended version (if we have time)
    - standard
        - normal rules
        - 2 human players
        - basic display (jupyter output)
    - extended
        - graphics display (turtle?)
        - hard mode ("Salvo")
        - computer player
            - random guess
            - if last guess == hit, random guess within closer range...
        - different size board
        - different size ships(?)


Standard:

- board
    - 10x10
    - 4 boards (player_1_sea, player_1_radar, player_2_sea, player_2_radar)

- ships
    - number/size has to make sense for the board dimensions
    - standard:
        - Carrier (5)
        - Battelship (4)
        - Cruiser (3)
        - Submarine (3)
        - Destroyer (2)

- turn
    - guess one coordinate on board
        - outputs result (miss / hit + ship if hit)
            - sunk if ship gone
        - displays new board (updated pegs)

- display
    - can only see your board

Functions:

- display_board
    - input lists
    - output (print) sea and radar

- player_names
    - input string

- place_ships
    - choose ship
    - choose starting coordinate
    - choose direction (udlr)
    - calls check_space_free (iterates for all coords in selection)
    - updates ship variables

- update board

- check_space_free

- choose_first_player
    - random (0, 1)

- fire_shot
    - choose location to fire at
    - update board/list

- check_hit
    - returns result of shot
    - update ship count

- check_ship_status
    - checks if 

- save_shot
    - remove available firing locations
    - decide whether can play in same square again

- ready_for_turn
    - when turn ended, hide board
    - ask if ready
    - when ready, show other player board

- winning_condition
    - ship count == 0

- replay

Game Logic

- initiate empty board
- ask for player names
- decide who goes first

- player 1 places ships
    - display_board per ship placed
    - ready_for_turn
- player 2 places ships
    - display_board per ship placed
    - ready_for_turn

- while not winning_condition:
    - player 1 fire_shot
        - display board
        - (update board)
        - check_hit
        - check_ship_status
        - check winning_condition
        - display board
        - ready_for_turn
    - player 2 fire_shot (same logic as player 1)

- display result
    - display both boards (?)

- ask for replay

## Functions

In [147]:
# Library imports
import random

In [126]:
# Initial variables

letters = ["A","B","C","D","E","F","G","H","I","J"]

# Radar squares
radar_empty = "   "
radar_boats = [" D ", " C ", " S ", " B ", " A "]
radar_hit = " x "
radar_miss = " o "

# Sea squares
sea_empty = "~~~"
sea_boats = ["~D~", "~C~", "~S~", "~B~", "~A~"]
sea_hit = "~x~"
sea_miss = "~o~"

# Boat types
boat_types = {"Aircraft carrier (A)" : 5, 
              "Battleship (B)" : 4, 
              "Submarine (S)" : 3, 
              "Cruiser (C)" : 3, 
              "Destroyer (D)" : 2}

# Lists of player boat locations
player_1_boats = []
player_2_boats = []

# To be included in game logic?
sea_lst_p1 = [sea_empty] * 100  
radar_lst_p1 = [radar_empty] * 100
sea_lst_p2 = [sea_empty] * 100  
radar_lst_p2 = [radar_empty] * 100

# Lists of past guesses
player_1_guesses = []
player_2_guesses = []

In [72]:
player_names()

type your name:Danielle
type your name:is the best


('Danielle', 'is the best')

In [73]:
def player_names():
    """Asks for the names of Players 1 and 2 and returns both in a tuple."""
    player_1 = input("type your name:")
    player_2 = input("type your name:")
    return player_1, player_2

def choose_first_player():
    """Randomly selects which player has the first turn. Returns 'p1' or 'p2'."""
    if random.randint(0,1) == 0:
        return "p1"
    else:
        return "p2"

def display_boards(radar_lst, sea_lst, letters):
    """Prints the radar_lst and sea_lst lists in the format of Battleships boards, with the letters list
    denoting the rows."""
    
    print("Radar zone\n")
    print("     1   2   3   4   5   6   7   8   9   10")
    for index, i in enumerate(radar_lst):
        if index % 10 == 0:
            letter = letters[index//10]
            print(f"{letter}: |{i}", end = "")
        elif index % 10 == 9:
            print(f"|{i}|")
        else:
            print(f"|{i}", end = "")
            
    print("\nSea zone\n")
    print("     1   2   3   4   5   6   7   8   9   10")
    for index, i in enumerate(sea_lst):
        if index%10 == 0:
            letter = letters[index//10]
            print(f"{letter}: |{i}", end = "")
        elif index % 10 == 9:
            print(f"|{i}|")
        else:
            print(f"|{i}", end = "")

In [69]:
practice = sea_lst_p1
practice[52] = sea_boats[3]
practice[62] = "\U0001F4A5"
practice[72] = sea_boats[3]
practice[82] = sea_boats[3]

display_boards(radar_lst_p1, practice, letters)

Radar zone

     1   2   3   4   5   6   7   8   9   10
A: |   |   |   |   |   |   |   |   |   |   |
B: |   |   |   |   |   |   |   |   |   |   |
C: |   |   |   |   |   |   |   |   |   |   |
D: |   |   |   |   |   |   |   |   |   |   |
E: |   |   |   |   |   |   |   |   |   |   |
F: |   |   |   |   |   |   |   |   |   |   |
G: |   |   |   |   |   |   |   |   |   |   |
H: |   |   |   |   |   |   |   |   |   |   |
I: |   |   |   |   |   |   |   |   |   |   |
J: |   |   |   |   |   |   |   |   |   |   |

Sea zone

     1   2   3   4   5   6   7   8   9   10
A: |~~~|~~~|~~~|~~~|~~~|~~~|~~~|~~~|~~~|~~~|
B: |~~~|~~~|~~~|~~~|~~~|~~~|~~~|~~~|~~~|~~~|
C: |~~~|~~~|~~~|~~~|~~~|~~~|~~~|~~~|~~~|~~~|
D: |~~~|~~~|~~~|~~~|~~~|~~~|~~~|~~~|~~~|~~~|
E: |~~~|~~~|~~~|~~~|~~~|~~~|~~~|~~~|~~~|~~~|
F: |~~~|~~~|~B~|~~~|~~~|~~~|~~~|~~~|~~~|~~~|
G: |~~~|~~~|💥|~~~|~~~|~~~|~~~|~~~|~~~|~~~|
H: |~~~|~~~|~B~|~~~|~~~|~~~|~~~|~~~|~~~|~~~|
I: |~~~|~~~|~B~|~~~|~~~|~~~|~~~|~~~|~~~|~~~|
J: |~~~|~~~|~~~|~~~|~~~|~~~|~~~|~~~|

In [17]:
def update_board(radar_lst, sea_lst, position, flag):
    """Taking the position and flag results from a turn, this function updates the radar_lst and 
    sea_lst lists that make up the board."""
    radar_lst[position] = flag
    sea_lst[position] = flag

In [80]:
def fire_shot(letters):
    
    guess_test = False
    while guess_test == False:
        guess = input("Enter coordinates to fire at: ").upper()
        if len(guess) >3:
            guess == input("Give a valid coordinate: ").upper()
        elif len(guess) == 3 and guess[1] != "1" and guess[2] != "0":
            guess == input("Give a valid coordinate: ").upper()
        else:
            try:
                if guess[0] in letters and int(guess[1]) in range(1,9):
                    for index,i in enumerate(letters):
                        position = letters.index(guess[0])*10 + int(guess[1:])
                        guess_test = True
                        return position
                else:
                    guess == input("Give a valid coordinate: ").upper()
            except ValueError:
                guess == input("Give a valid coordinate: ").upper()

In [24]:
fire_shot(letters)

Enter coordinates to fire at: G4


64

In [137]:
def enforce_player_coord(coord):
    """Ensures a player coordinate input is in the valid 'A1' - 'J10' format. Returns coord once in the correct format."""
    # Ensure second item in coord is a number
    try:
        int(coord[1])
    except (ValueError, IndexError):
        coord = input("Enter coordinates (in the format 'A1'): ").upper()

    # Ensure coord is 2-3 characters long (and ends in '10') if 3 characters, starts with a letter in letters and
    # ends in a number 1-10
    while len(coord) > 3 or len(coord) < 2 or \
    (len(coord) == 3 and coord[1:3] != "10") or \
    coord[0] not in letters or int(coord[1]) not in range(1, 9):
        coord = input("Enter coordinates (in the format 'A1'): ").upper()
     
    return coord

In [138]:
def test_fire(player_guesses, letters):
    """Calls enforce_player_coord() to prompt player to enter a guess and enforce validity.
    If coordinates have been guessed by the player already, asks for another guess.
    If the guess is valid, adds the guess to player_guesses and returns the 
    list index ('position') of the player guess."""
    
    guess = ""
    guess = enforce_player_coord(guess)
    
    # Checks if guess has already been guessed in a previous turn
    while guess in player_guesses:
        print("Coordinates already guessed!")
        guess = input("Enter coordinates (in the format 'A1'): ").upper() 
    
    player_guesses.append(guess) # adds guess to guess history
    position = letters.index(guess[0]) * 10 + int(guess[1:])
    return position

In [139]:
test_guesses = ["A5"]

position = test_fire(test_guesses, letters)

print(test_guesses)
print(position)

Enter coordinates (in the format 'A1'): asd
Enter coordinates (in the format 'A1'): 
Enter coordinates (in the format 'A1'): A5
Coordinates already guessed!
Enter coordinates (in the format 'A1'): G6
['A5', 'G6']
66


In [136]:
def check_space_free(sea_lst, start_coord, direction):
    """Checks if all required spaces for boat placement are free and in range.
    Returns coordinates if valid, otherwise returns False."""
    pass

In [151]:
def place_boats(sea_lst, player_boats):
    """Prompts user to place boats in descending order of length, by entering starting co-ordinate and direction.
    Calls check_space_free() to ensure boat placement is valid - if not, prompts user to place boat elsewhere.
    Saves placed boats in player_boats list."""
    
    for boat, length in boat_types.items():
        coords = ""
        while not coords:
            start_coord = enforce_player_coord(input(f"Enter start coordinate for {boat}: "))
            direction = ""
            while direction not in ["U","D","L","R"]:
                direction = input("Choose direction (U/D/L/R): ").upper()
            coords = check_space_free(sea_lst, start_coord, direction)
        player_boats.append(coords)

In [149]:
# place_boats(practice, )

In [58]:
def check_hit(sea_lst, position):
    """Takes player guess position and returns True if that position in sea_lst was a boat."""
    if sea_lst[position] in sea_boats:
        return True       
    else:
        return False

In [60]:
test_guess = 72

check_hit(practice, test_guess)

True

In [93]:
def check_ship_status():
    """Checks which ships are destroyed and updates ship count."""
    pass

# Have running score on side?

In [97]:
def winning_condition():
    """Returns true if either player has 0 ships remaining."""
    pass

In [94]:
def new_turn():
    """When turn ended (i.e. function is called), hides the previous player's board and asks next player if they are ready.
    Once second player is ready, sets turn to their go and displays their board."""
    pass

In [95]:
def final_result():
    """Displays both boards at the end of the game."""
    pass

In [96]:
def replay():
    """Asks players if they want to play again. If 'Yes', restarts game. If 'No', exits program."""
    pass

In [65]:
print("\U0001F6E5")
print("\U000026F4")
print("\U0001F4A5")

🛥
⛴
💥
